### Feature Extraction
* **Products used:** 
[dem_cop_30](https://explorer.digitalearth.africa/products/s2_l2a), [s2_l2a](https://explorer.digitalearth.africa/products/dem_cop_90), [dem_srtm](https://explorer.digitalearth.africa/products/dem_srtm), [dem_srtm_deriv](https://explorer.digitalearth.africa/products/dem_srtm_deriv)

## Background:

Training data extraction plays a crucial role in training machine learning models. The process involves extracting relevant feature layers from a geospatial dataset based on predefined geometries or regions of interest. This enables the creation of accurate and reliable classification models for various applications such as land cover mapping, crop monitoring, and environmental analysis.

To facilitate this task, the open-data-cube provides a powerful function called "collect_training_data." This function is part of the deafrica_tools.classification script and is specifically designed to extract training data from the open-data-cube using geometries defined within a GeoJSON file. The GeoJSON file contains the spatial boundaries or polygons that delineate the regions of interest for which training data needs to be extracted.

## Description:

This notebook focuses on the extraction of training data (feature layers) from the open-data-cube using geometries defined within a GeoJSON file. It follows a step-by-step approach to guide users in utilizing the "collect_training_data" function effectively. The goal is to enable users to extract the appropriate training data for their specific use case.

The main steps in this notebook are as follows:

1. **Previewing the Training Data:** The notebook starts by plotting the polygons from the training data on a basemap. This visualization provides users with a visual representation of the regions of interest for which training data will be extracted.

2. **Defining the Feature Layer Function:** Next, a feature layer function is defined. This function specifies the set of feature layers to be extracted from the open-data-cube. These layers are carefully selected based on their relevance to the classification task at hand.

3. **Extracting Training Data:** The "collect_training_data" function is then employed to extract the training data from the datacube. It utilizes the predefined geometries from the GeoJSON file and retrieves the corresponding feature layers. This step ensures that the extracted data aligns precisely with the defined regions of interest.

4. **Exporting Training Data:** Finally, the extracted training data is exported and saved to disk. This facilitates its subsequent use in other scripts or machine learning workflows for training classification models.

By following the steps outlined in this notebook, users can leverage the "collect_training_data" function to efficiently extract training data from the open-data-cube. 

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [ ]:
%matplotlib inline
import io
import os 
import math
import datacube
import warnings
import rioxarray
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from odc.io.cgroups import get_cpu_quota
from datacube.testutils.io import rio_slurp_xarray


from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.bandindices import calculate_indices
from classification import collect_training_data
from externaldrive import list_gdrive, read_tif_from_gdrive

## Analysis parameters
 * path: The path to the input vector file from which we will extract training data. A default geojson is provided.
 * field: This is the name of column in your shapefile attribute table that contains the class labels. The class labels must be integers

In [ ]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
prefix = 'aoi'
field = 'class_id'
path = f'data/{prefix}_training_samples.geojson'

# Load input data shapefile
training_points= gpd.read_file(path) 
training_points.head()

In [ ]:
# Set a flag to convert to polygons:
use_polygons = True

if use_polygons:
    # Convert from lat,lon to EPSG:6933 (projection in metres)
    training_points = training_points.to_crs("EPSG:6933")

    # Buffer geometry to get a square - only if trying to sample multiple pixels
    buffer_radius_m = 10
    training_points.geometry = training_points.geometry.buffer(buffer_radius_m, cap_style=3)

#### Plot on interactive map 

In [ ]:
points = training_points
training_points.explore(
    tiles = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}", 
    attr ='Imagery @2022 Landsat/Copernicus, Map data @2022 Google',
    popup=True,
    cmap='viridis',
    style_kwds=dict(radius= 5, color= 'red', fillOpacity= 0.8, fillColor= 'red', weight= 3),
    )

## Defining Query

The function `collect_training_data` takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any `NaN` or `inf` values.

The below variables can be set within the `collect_training_data` function:

* `field`: The name of column in your geojson file attribute table that contains the class labels, which corresponds to the `class_attr` that we defined earlier.
* `zonal_stats`: An optional string giving the names of zonal statistics to calculate across each geometry (polygon or point). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.
* `dc_query`: A datacube query dictionary for the Open Data Cube query such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). 
* `feature_func`:  A function for generating feature layers that is applied to the data within the bounds of the input geometry. This function will take the 'dc_query' as the only argument.
* `return_coords`: If True, then the training data will contain two extra columns ‘x_coord’ and ‘y_coord’ corresponding to the x,y coordinate of each sample.

> Note: `collect_training_data` also has a number of additional parameters for handling ODC I/O read failures, where polygons that return an excessive number of null values can be resubmitted to the multiprocessing queue.  Check out the [docs](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/83116e80ebb4f8744e3de74e7a713aadd0a7577a/Tools/deafrica_tools/classification.py#L565) to learn more.

We will define the first three parameters and describe the `feature_func` seperately in a moment.

In [ ]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
time = ('2024')

resolution = (-10,10)

output_crs='epsg:6933'

Note that we've selected nine spectral bands with spatial resolution no lower than 20 m here for demonstration. However, it is advised that you test and select the bands based on your own classification task. Using the variables above, we can generate a datacube query object from the parameters above:

In [ ]:
query = {
    'time': time,
    'output_crs': output_crs,
    'resolution': resolution,
}

### Defining feature function

## Defining feature function

To create the desired feature layers, we pass instructions to `collect_training_data` through the `feature_func` parameter. The `feature_func` must accept a `dc_query` dictionary, and return a single `xarray.Dataset` or `xarray.DataArray` containing 2D coordinates (i.e x, y - no time dimension). e.g.

          def feature_function(query):
              dc = datacube.Datacube(app='feature_layers')
              ds = dc.load(**query)
              ds = ds.mean('time')
              return ds

Below, we will define a more complicated feature layer function than the brief example shown above. Firstly We will calculate the Normalised Difference Water Index (NDWI), which is commonly used to distinguish between Water and non-water land cover classes. We use the `calculate_indices`function to automatically calculate NDVI for all specified bands. 

In addition, we'll use temporal signatures to help distinguish wetland classes. To reduce data size while keeping seasonal changes, we are implementing biannual temporal aggregation, i.e. geomedian (sometimes referred to as the 'geometric median') for each pixel location.

### Read terrain indices from a Google Drive Folder. 
Make sure you have followed the instructions to set up the connection with [Google Drive API using a service account](https://docs.digitalearthafrica.org/en/latest/platform_tools/googledrive_access.html). This code should only be used when the terrain attribute data is located in a Google Drive to save on Sandbox disk space.

In [ ]:
# # Capture the list of TIFF files from Google Drive
# tif_files = list_gdrive()

# # Initialize an empty dataset for merging
# terrain_stacked = xr.Dataset()

# # Ensure tif_files contains the expected structure
# if isinstance(tif_files, list):
#     # Filter the TIFF files from the list
#     tif_files = [file for file in tif_files if file['name'].endswith('.tif')]

#     # Display the TIFF files with their IDs
#     if tif_files:
#         print("Available TIFF files:")
#         for tif in tif_files:
#             print(f"{tif['name']} (ID: {tif['id']})")

#         # Lists to store the data arrays and their extents
#         data_arrays = []
#         extents = []
#         titles = []

#         # Read and merge all the TIFF files into the dataset
#         for tif in tif_files:
#             selected_file_id = tif['id']  # Select the current TIFF file
            
#             # Read the selected TIFF file from Google Drive
#             data_array, transform = read_tif_from_gdrive(selected_file_id)

#             # Check if the data was read successfully
#             if data_array is not None:
#                 print(f"Data for {tif['name']} read successfully!")

#                 # Convert to a dataset with the filename as the variable name
#                 tif_dataset = data_array.to_dataset(name=tif['name'].replace('.tif', ''))

#                 # Merge with the existing stacked dataset
#                 terrain_stacked = xr.merge([terrain_stacked, tif_dataset], compat='override')

#                 # Store the data array and its extent
#                 data_arrays.append(data_array)
#                 x_min, x_max = transform[2], transform[2] + transform[0] * data_array.shape[1]
#                 y_min, y_max = transform[5] + transform[4] * data_array.shape[0], transform[5]
#                 extents.append((x_min, x_max, y_min, y_max))
#                 titles.append(tif['name'])  # Store the title for the plot

#             else:
#                 print(f"Failed to read data for {tif['name']}.")

#         # # Plot all TIFF data in subplots after reading them all
#         # # Calculate the number of rows needed
#         # num_files = len(data_arrays)
#         # num_columns = 4
#         # num_rows = math.ceil(num_files / num_columns)
        
#         # # Create subplots with the desired number of rows and columns
#         # fig, axes = plt.subplots(nrows=num_rows, ncols=num_columns, figsize=(15, 5 * num_rows))

#         # # Flatten the axes array for easier indexing
#         # axes = axes.flatten()
        
#         # # Loop through the files and plot them
#         # for i in range(num_files):
#         #     im = axes[i].imshow(data_arrays[i], cmap='gray', extent=extents[i])
#         #     axes[i].set_title(titles[i])
#         #     axes[i].set_xlabel('X Coordinate')
#         #     axes[i].set_ylabel('Y Coordinate')
        
#         #     # Add a color bar to each subplot
#         #     cbar = fig.colorbar(im, ax=axes[i], orientation='vertical', fraction=0.046, pad=0.04)
#         #     cbar.set_label('Pixel Value')
        
#         # # Hide axes for unused subplots if any
#         # for i in range(num_files, len(axes)):
#         #     axes[i].axis('off')
        
#         # # Adjust layout to prevent overlap
#         # plt.tight_layout()
#         # plt.show()

#         # Print a summary of the final merged dataset
#         print(f"Final stacked dataset contains {len(terrain_stacked.data_vars)} variables.")
#     else:
#         print("No TIFF files found.")
# else:
#     print("Failed to retrieve files from Google Drive.")

In [ ]:
def feature_layers(query, terrain_stacked=None):
    dc = datacube.Datacube(app='feature_layers')
    
    # --- Sentinel-2 Annual Geomedian and Indices ---
    ds = dc.load(
        product='gm_s2_annual',
        measurements=['blue', 'green', 'red', 'nir_1', 'nir_2', 'swir_1', 'swir_2', 'emad', 'smad', 'bcmad'],
        **query
    )
    ds = calculate_indices(ds, index=['NDVI', 'MNDWI', 'TCW'], drop=False, satellite_mission='s2')
    keep_vars = ['NDVI', 'MNDWI', 'TCW', 'emad', 'smad', 'bcmad']
    ds = ds[keep_vars]
    ds = ds.rename({var: f'Annual_{var}' for var in ds.data_vars})
    
    # Stack temporal layers
    stacked_vars = []
    for var in ds.data_vars:
        for i in range(ds.sizes['time']):
            stacked_vars.append(ds[var].isel(time=i).rename(f'{var}_{i}'))
    ds_stacked = xr.merge(stacked_vars, compat='override')

    # --- Sentinel-1 monthly mosaic ---
    ds_s1 = dc.load(product="s1_monthly_mosaic", measurements=['vv', 'vh'], group_by="solar_day", **query)
    ds_s1 = ds_s1.rename({v: f"sentinel-1_{v}" for v in ds_s1.data_vars})
    median_s1 = ds_s1.median(dim='time')

    # --- ALOS PALSAR ---
    ds_alos = dc.load(product='alos_palsar_mosaic', measurements=['hh', 'hv'], **{**query, 'time': ('2022')})
    ds_alos = ds_alos.rename({v: f"alos_palsar_{v}" for v in ds_alos.data_vars})
    median_alos = ds_alos.median()

    # --- WOfS ---
    wofs_annual = dc.load(product='wofs_ls_summary_annual', like=ds.geobox, time=query['time'])
    wofs_annual_freq = wofs_annual.frequency.squeeze('time', drop=True).rename('WOfS_annual')

    wofs_alltime = dc.load(product='wofs_ls_summary_alltime', like=ds.geobox)
    wofs_alltime_freq = wofs_alltime.frequency.squeeze('time', drop=True).rename('WOfS_alltime')

    # --- CHIRPS Rainfall ---
    chirps = dc.load(product='rainfall_chirps_monthly', like=ds.geobox, time=query['time'])
    chirps_mean = chirps.rainfall.mean(dim='time').rename('Chirps_rainfall')

    # --- Cropland Probability ---
    crop_prob = dc.load(product='crop_mask', like=ds.geobox).prob.rename('Crop_probability')

    # --- Land Cover Dataset - ESA Worldcover ---
    esa_lulc = dc.load(product='esa_worldcover_2021', measurements='classification', like=ds.geobox)
    esa_lulc = esa_lulc.classification.squeeze('time', drop=True).rename('ESA_2021_lulc')


    # Option 1: Use this code if the terrain attribute files are located in a folder within the Sandbox.
    # Uncomment the code below and comment out the Google Drive option if using this method.
    # loop through the terrain attribite files and add them to the dataset
    
    folder = os.path.join("data/terrain_attributes/", prefix)
    for filename in os.listdir(folder):
            if filename.endswith('.tif'):
                filepath = os.path.join(folder, filename)
                tif = rio_slurp_xarray(filepath, gbox=ds.geobox)
                tif = tif.to_dataset(name=filename.replace('.tif', ''))
                ds_stacked = xr.merge([ds_stacked, tif], compat='override')
                
    # Option 2: Use this code if the terrain attribute files are in a Google Drive folder
    # Uncomment the code below and comment out the Sandbox folder option above if using this method.
    # Make sure to change None to terrain_stacked in the function argument at the top if using terrain indices from a google drive folder

    # bbox = ds.geobox.extent.boundingbox
    # if terrain_stacked is not None:  
    #     terrain_stacked.attrs['crs'] = ds.geobox.crs
    #     terrain_stacked = terrain_stacked.sel(x=slice(bbox.left, bbox.right), y=slice(bbox.top, bbox.bottom))
    #     terrain_stacked = terrain_stacked.rio.reproject_match(ds)
    #     ds_stacked = xr.merge([ds_stacked, terrain_stacked], compat='override', combine_attrs='override')
    
    # --- Merge All Layers ---
    ds_stacked = xr.merge([
        ds_stacked, median_s1, median_alos,
        wofs_annual_freq, wofs_alltime_freq,
        chirps_mean, crop_prob, esa_lulc
    ], compat='override', combine_attrs='override')
    
    return ds_stacked


Now let's run the `collect_training_data` function. This may take minutes to hours depending on your number of training points, number of measurements/bands set for the query and the calculation work in the feature function. Since we've used 10 measurements (9 spectral bands and 1 NDWI index) with 6 temporal geomedian for each band, it can be very time-consuming to finish the training features extraction. Therefore, here we are passing in `gdf=training_points[0:10]` to only run the code over the first 10 geometries as demonstration. Nevertheless, the extracted full training data file is provided in the 'Results/' folder, which will be used for next module of the workflow.

> **Note**:  With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization. However, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization.

In [ ]:
##### detect the number of CPUs
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

# Ensure the 'field' column is of integer dtype
training_points[field] = training_points[field].astype(int)

# collect training data
column_names, model_input = collect_training_data(
    gdf=training_points[0:1],
    dc_query=query,
    ncpus=1,
    field=field, # integer class label
    zonal_stats="mean",
    feature_func=feature_layers,
    return_coords=True)

In [ ]:
print(column_names)

In [ ]:
pd_training_features=pd.DataFrame(data=model_input,columns=column_names)
pd_training_features

### Export training features

In [ ]:
# convert the data to geopandas dataframe
pd_training_features=pd.DataFrame(data=model_input,columns=column_names)
#set the name and location of the output file
# output_file = "results/training_features.txt"
output_file = f"results/{prefix}_training_features.txt"
#Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=' ')

In [ ]:
# create geopandas dataframe
gpd_training_features=gpd.GeoDataFrame(pd_training_features, 
geometry=gpd.points_from_xy(model_input[:,-2], model_input[:,-1],crs=output_crs))

#####  Add a column for binary (wetland/non-wetland) classification
This block ensures that both binary and multi-class classification labels are properly set up from the original `class_id` field:

- If `class_id` contains only two values (0 and 1), it is assumed to be binary, and is renamed to `class_id_binary`.
- If `class_id` includes additional wetland types, a new binary column `class_id_binary` is created:
  - `1` for any wetland type (i.e., values not equal to 0)
  - `0` for non-wetland (i.e., value equal to 0)
  - The original column is then renamed to `class_id_type` for use in multi-class classification.

In [ ]:
# Check if unique values in 'class_id' are only 0 and 1
unique_values = gpd_training_features['class_id'].unique()
if len(unique_values) == 2 and set(unique_values) == {0, 1}:
    # Replace 'class_id' with 'class_id_binary'
    gpd_training_features.rename(columns={'class_id': 'class_id_binary'}, inplace=True)
else:
    # Create 'class_id_binary' column based on condition
    gpd_training_features['class_id_binary'] = gpd_training_features['class_id'].apply(lambda x: 1 if x != 0 else 0)
    gpd_training_features.rename(columns={'class_id': 'class_id_type'}, inplace=True)

# Insert the new column at the second position
gpd_training_features.insert(0, 'class_id_binary', gpd_training_features.pop('class_id_binary'))
print(gpd_training_features.columns)

In [ ]:
# Replace non-zero values in the 'class_id' column with 1
gpd_training_features['class_id_binary'] = gpd_training_features['class_id_type'].apply(lambda x: 1 if x != 0 else 0)
# Insert the new column at the second position
gpd_training_features.insert(1, 'class_id_binary', gpd_training_features.pop('class_id_binary'))
gpd_training_features

In [ ]:
# save as geojson file
# gpd_training_features.to_file('results/training_features.geojson', driver="GeoJSON")
geojson_file = f"results/{prefix}_training_features.geojson"
gpd_training_features.to_file(geojson_file, driver="GeoJSON")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')